# Step39, 합계함수 

## 39.1 sum 함수의 역전파 

In [1]:
import numpy as np 
from dezero import Variable, Function
import dezero.functions as F

x = Variable(np.array([1,2,3,4,5,6]))
y = F.sum(x)
y.backward()
print(y)
print(x.grad)

variable(21)
variable([1 1 1 1 1 1])


In [2]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sum(x)
y.backward()
print(y)
print(x.grad)

variable(21)
variable([[1 1 1]
          [1 1 1]])


## 39.2 sum 함수 구현 

DeZero의 sum 함수 역전파에서는  
입력 변수의 shape과 같아지도록 기울기의 원소를 복사한다.  
그런데 역전파에서는 Variable 인스턴스를 사용하므로 복사 작업도 DeZero 함수로 해야한다.

**CAUTION_** 지정한 shape에 맞게 원소를 복사하는 작업은 넘파이의 브로드캐스트와 같은 기능

다음단계에서 구현할 broadcast_to 함수를 미리 사용한다.

~~~python
class Sum(Function):
    def forward(self, x):
        self.x_shape = x.shape 
        y = x.sum()
        return y 
    def backward(self, gy):
        gx = broadcast_yo(gy, self.x_shape)
        return gx 

def sum(x):
    return Sum()(x)
~~~

In [3]:
import numpy as np 
from dezero import Variable
import dezero.functions as F

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sum(x)
y.backward()
print(y)
print(x.grad)

variable(21)
variable([[1 1 1]
          [1 1 1]])


## 39.3 axis와 keepdims 

합계를 구할 때 '축'을 지정할 수 있다.

In [4]:
x = np.array([[1,2,3],[4,5,6]])
y = np.sum(x, axis=0)
print(y)
print(x.shape, '->', y.shape)
y = np.sum(x, axis=1)
print(y)
print(x.shape, '->', y.shape)

[5 7 9]
(2, 3) -> (3,)
[ 6 15]
(2, 3) -> (2,)


axis=0은 세로    
axis=1은 가로

In [5]:
x = np.array([[1,2,3],[4,5,6]])
y = np.sum(x, keepdims=True)
print(y)
print(x.shape, '->', y.shape)

[[21]]
(2, 3) -> (1, 1)


Dezero의 sum 함수에 axis와 keepdims 인수를 지원하도록 수정한다.

In [6]:
# functions.py 
from dezero import utils

class Sum(Function):
    def __init__(self, axis, keepdims):
        self.axis = axis
        self.keepdims = keepdims

    def forward(self, x):
        self.x_shape = x.shape
        y = x.sum(axis=self.axis, keepdims=self.keepdims) 
        return y 
    
    def backward(self, gy):
        gx = utils.reshape_sum_backward(gy, self.x_shape, self.axis, self.keepdims)
        gx = broadcast_to(gy, self.x_shape)
        return gx

def sum(x, axis=None, keepdims=False):
    return Sum(axis, keepdims)(x)

In [7]:
x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sum(x, axis=0)
y.backward()
print(y)
print(x.grad)

variable([5 7 9])
variable([[1 1 1]
          [1 1 1]])


In [8]:
x = Variable(np.random.rand(2,3,4,5))
y = x.sum(keepdims=True)
print(y.shape)

(1, 1, 1, 1)
